______
## Species Ingest

In [15]:
import sqlite3

# Database standard connection
conn=sqlite3.connect('nursery.db')
cursor=conn.cursor()

# Premilinar species data
species_data =     [
    ('Tabebuia rosea', 'Roble Rosado', 'Bignoniaceae', '2025-05-18', 'De 8 a 10 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Tabebuia impetiginosa', 'Roble Morado', 'Bignoniaceae', '2025-05-02', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Laburnum anagyroides', 'Lluvia de Oro', 'Fabaceae', '2025-06-22', 'De 8 a 10 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla-esqueje'),
    ('Mangifera indica', 'Mango', 'Anacardiaceae', '2025-07-06', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Injerto-semilla'),
    ('Delonix regia', 'Acacia', 'Fabaceae', '2025-05-18', 'De 8 a 10 días', 'Rápido', 'Medio', 'Sombra parcial', 'Semilla'),
    ('Citrus × paradisi', 'Toronja', 'Rutaceae', '2025-05-11', 'Más de 15 días', 'Lento', 'Medio', 'Pleno sol', 'Injerto-semilla'),
    ('Carica papaya', 'Papaya', 'Caricaceae', '2025-05-11', 'Menos 8 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dypsis lutescens', 'Palma Areca', 'Arecaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Sombra parcial', 'División-semilla'),
    ('Vachellia farnesiana', 'Aromo', 'Fabaceae', '2025-08-29', 'De 10 a 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dracaena trifasciata', 'Lengua de Suegra', 'Asparagaceae', '2025-08-11', 'Más de 15 días', 'Lento', 'Bajo', 'Pleno sol', 'División-esqueje'),
    ('Terminalia catappa', 'Almendro', 'Combretaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Zamioculcas zamiifolia', 'Millonaria', 'Araceae', '2025-08-29', 'De 10 a 15 días', 'Lento', 'Bajo', 'Sombra parcial', 'División-esqueje'),
    ('Luffa aegyptiaca', 'Estropajo', 'Cucurbitaceae', '2025-08-11', 'De 8 a 10 días', 'Rápido', 'Alto', 'Pleno sol', 'Semilla'),
    ('Cassia grandis', 'Cañandonga', 'Fabaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dracaena marginata', 'Marginata Tricolor', 'Asparagaceae', '2025-09-10', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Esqueje'),
    ('Tradescantia pallida', 'Purpurina', 'Commelinaceae', '2025-09-10', 'Menos 8 días', 'Moderado', 'Bajo', 'Pleno sol', 'Esqueje'),
    ('Tradescantia spathacea', 'Maguey morado', 'Commelinaceae', '2025-09-19', 'De 8 a 10 días', 'Lento', 'Bajo', 'Sombra parcial','División-esquje'),
    ('Plumeria', 'Bouquet de novia', 'Apocynaceae', '2025-09-19', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol','Semilla-esqueje'),
    ('Persea americana', 'Aguacate', 'Lauraceae', '2025-09-20', 'Más de 15 días', 'Rápido', 'Alto', 'Pleno sol', 'Semilla-esqueje'),
    ('Pasiflora edulis', 'Maracuyá', 'Passifloraceae', '2025-09-19', 'Más de 15 días', 'Rápido', 'Medio', 'Sombra parcial', 'Semilla'),
    ('Solanum quitoense', 'Lulo', 'Solanaceae', '2025-09-19', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla-esqueje')
]

# parametrized query for multiple row insertion
insert_query = '''
    INSERT INTO ESPECIE_LOTE
        (NOMBRE_CIENTIFICO, NOMBRE_COMUN, FAMILIA, FECHA_INICIO, GERMINACION_PROMEDIO, CRECIMIENTO, RIEGO, LUZ, PROPAGACION)
    VALUES
        (?,?,?,?,?,?,?,?,?)
'''

# Excecute in transaction
with sqlite3.connect('nursery.db') as conn:
    conn.executemany(insert_query, species_data)

In [ ]:
import sqlite3

def insert_specie(datos_especie):
    """
    Insert a new species to the ESPECIE_LOTE table

    Parameters:
        datos_especie: Tuple with the data to import acording to the table
    """
    try:
        # Connect to the database
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()

        insert_query = ''' 
        INSERT INTO ESPECIE_LOTE 
        (NOMBRE_CIENTIFICO, NOMBRE_COMUN, FAMILIA, FECHA_INICIO, 
        GERMINACION_PROMEDIO, CRECIMIENTO, RIEGO,LUZ, PROPAGACION, NOTAS)
        VALUES (?,?,?,?,?,?,?,?,?,?)
        '''
        # Execute the insertion
        cursor.execute(insert_query, datos_especie)
        conn.commit()

        print("Especie insertada correctamente!")

    except sqlite3.Error as error:
        print(f"Error al insertar: {error}")
    
    finally:
        # close connection
        if conn:
            conn.close()

# New species data
new_data = (
    "Solanum quitoense", # NOMBRE_CIENTIFICO
    "Lulo",  # NOMBRE_COMÚN
    "Solanaceae",     # FAMILIA
    "2025-09-19",      # FECHA_INICIO
    "Más de 15 días",  # GERMINACIÓN_PROMEDIO
    "Moderado",          # CRECIMIENTO (Rápido, Moderado, Lento)
    "Medio",           # RIEGO (Alto, Medio, Bajo)
    "Pleno sol",       # LUZ (Sombra parcial, Pleno sol)
    "Semilla-esqueje", # PROPAGACIÓN (Semilla, Esqueje, División, Acodo o injerto)
    "Brote y división de una toronja" # NOTAS
)
    
insert_specie(new_data)

Especie insertada correctamente!


_______
# Irrigation Ingest

In [1]:
import sqlite3
from datetime import date

def log_daily_irrigation(irrigation_list, log_date=None):
    """
    A reusable function to log daily irrigation for MULTIPLE lots at once.
    
    Parameters:
    irrigation_list (list of tuples): Each tuple is (lote_id, water_lt, notes).
    log_date (str, optional): Date in 'DD-MM-YYYY' format. Uses today if not provided.
    """
    # Use today's date if no specific date is provided
    if log_date is None:
        log_date = date.today().strftime("%d-%m-%Y")
    
    conn = sqlite3.connect('nursery.db')
    cursor = conn.cursor()
    
    success_count = 0
    total_water = 0.0
    
    try:
        # Calculate total water first
        total_water = sum(water_lt for (lote_id, water_lt, notes) in irrigation_list)
        
        # Prepare the query
        insert_query = '''
            INSERT INTO REGISTRO_RIEGO (ESPECIE_LOTE_ID, FECHA, CANTIDAD_AGUA_LT, NOTAS)
            VALUES (?, ?, ?, ?)
        '''
        
        # Prepare the data: add the same date to every entry
        data_to_insert = [(lote_id, log_date, water_lt, notes) for (lote_id, water_lt, notes) in irrigation_list]
        
        # Execute all inserts at once
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()
        
        success_count = len(irrigation_list)
        print(f"✅ Successfully logged irrigation for {success_count} lots on {log_date}.")
        print(f"💧 Total water used: {total_water} liters")
        
    except sqlite3.Error as e:
        print(f"❌ An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()
    
    return success_count, total_water

# --- HOW TO USE IT DAILY ---
# 1. Define your irrigation for the day in this list
# Format: (lote_id, water_amount, notes)
today_irrigation = [
    (1, 0.5, None),
    #(2, 5,None),
    (3, 2, None),
    #(4, 2, None),
    #(5, 0.5, None),
    #(6, 2, None),
    #(7, 1, None),
    (8, 0.5, None),
    #(9, 3, None),
    #(10, 0.5, None),
    (11, 0.5, None),
    #(13, 0.5 , None),
    #(14, 0.5, None),
    (15, 2, None),
    #(16, 1, None),
    (17, 3, None),
    #(18, 5, None)
    # Add more lots as needed...
]

# 2. Run this single function call
log_daily_irrigation(today_irrigation)

# --- FOR BACKFILLING PAST DATES ---
# If you need to log for a different date, just provide the date:
# last_weeks_irrigation = [(3, 4.0, ''), (4, 1.5, '')]
# log_daily_irrigation(last_weeks_irrigation, log_date='25-08-2025')

✅ Successfully logged irrigation for 6 lots on 24-09-2025.
💧 Total water used: 8.5 liters


(6, 8.5)

________
#  Batch Observation

In [ ]:
import sqlite3
from datetime import date

# Create a function to insert periodical observation data

def log_observation (observations, observation_date=None):
    """ 
    Function for periodical observation for MULTIPLE lots and count follow-up

    Parameters: 
    observation (list of tuples): Each tuple for all observation fields (lote_id, date, plant_qt, height, stage, health, notes)
    date (str): Date in DD-MM-YYYY format. Uses today if not provided.
    """

    # Use today's date if not specific date is provided
    if observation_date is None:
        observation_date = date.today().strftime("%d-%m-%y")

    try:
        # Connect to the database
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()

        # Calculate total plant count
        total_plants = sum(plant_qt for (lote_id, plant_qt, height, stage, health, notes) 
                           in observations)

        # Prepare the query
        insert_query = ''' 
            INSERT INTO OBSERVACION_LOTE (ESPECIE_LOTE_ID, FECHA_OBSERVACION, CANTIDAD_PLANTAS_VIVAS, 
            ALTURA_PROMEDIO_CM, ETAPA_CRECIMIENTO, INDICADOR_SALUD, NOTAS)
            VALUES (?,?,?,?,?,?,?)
        '''

        # Prepare the data: add the same date to every entry
        data_to_insert =  [(lote_id, observation_date, plant_qt, height, stage, health, notes) 
                           for (lote_id, plant_qt, height, stage, health, notes) in observations]

        # Excecute all inserts at once
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()

        print(f"✅Successfully logged {len(observations)} observations")
        print(f"📊 Total living plants")

    except sqlite3.Error as e:
        print(f"❌ An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()
    
    return total_plants
        
    
# --- HOW TO USE IT DAILY ---
if __name__ == "__main__":
    # 1. Define your observations for the day in this list
    # Format: (lote_id, plant_qt, height, stage, health, notes)
    today_observations = [
        
        # Add more lots as needed...
        # (1, 0, 0, '', '', ''),
        (2, 100, 38, 'PRE-EMBOLSE', 'Buena', 'Se introdujo mejor drenaje a las bolsas y la zona'),
        (3, 88, 33, 'PRE-EMBOLSE', 'Buena', 'Necesidad de atender 2 en cuarentena por palometa'),
        (4, 39, 15, 'CRECIMIENTO_VEGETATIVO', 'Buena', 'Sin Observaciones'),
        # (5, 0, 0, '', '', ''),
        # (6, 0, 0, '', '', ''),
        (6, 76, 20, 'CRECIMIENTO_VEGETATIVO', 'Buena', 'Sin Observaciones'),
        (7, 16, 30, 'ENDURECIMIENTO', 'Buena', 'Al menos seis están listas para sembrar'),
        # (8, 0, 0, '', '', ''),
        # (9, 0, 0, '', '', ''),
        # (10, 0, 0, '', '', ''),
        # (11, 0, 0, '', '', ''),
        # (12, 0, 0, '', '', ''),
        # (13, 0, 0, '', '', ''),
        # (14, 0, 0, '', '', ''),
        # (15, 0, 0, '', '', ''),
        # (16, 0, 0, '', '', ''),
    ]


    # 2. Run this single function call
    result = log_observation(today_observations)
    
    if result is not None:
        print(f"Operation completed successfully. Total plants: {result}")
    else:
        print("Operation failed.")

    # --- FOR BACKFILLING PAST DATES ---
    # If you need to log for a different date, just provide the date:
    # last_week_observations = [
    #     (3, 60, 22, 'Plantula embolsada', 'Regular', 'Necesita más agua'),
    #     (4, 55, 19, 'Plantula embolsada', 'Buena', '')
    # ]
    # log_observation(last_week_observations, observation_date='25-08-2024')

✅Successfully logged 5 observations
📊 Total living plants
Operation completed successfully. Total plants: 319


______
# Plant Entrance or Addition to the Nursery

In [8]:
import sqlite3
from datetime import date

def log_entrance(entrances, entrance_date=None):
    """ 
    Function for inserting multiple plant entrances with automated CODIGO_ENTRADA generation

    Parameters:
    entrances (list of tuples): Each tuple contains all observation fields except CODIGO_ENTRADA
    entrance_date (str): Date in DD-MM-YY format. Uses today if not provided.
    """

    # Use today's date if no specific date is provided
    if entrance_date is None:
        entrance_date = date.today().strftime("%d-%m-%y")

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()
        
        # Generate CODIGO_ENTRADA for each entrance and prepare data for insertion
        data_to_insert = []
        for entrance in entrances:
            especie_lote_id = entrance[0]
            
            # Count existing entries for this species and date to determine consecutive number
            # NOTE: Fixed the parentheses and added comma to make single parameter a tuple
            cursor.execute('''
                SELECT COUNT(*) FROM ENTRADA_PLANTAS 
                WHERE ESPECIE_LOTE_ID = ? AND FECHA_ENTRADA = ?
            ''', (especie_lote_id, entrance_date))  # Fixed: proper tuple with comma

            existing_count = cursor.fetchone()[0]
            consecutive_number = existing_count + 1

            # Generate unique CODIGO_ENTRADA
            codigo_entrada = f"{especie_lote_id}_{entrance_date}_{consecutive_number:02d}"

            # Reconstruct the tuple with CODIGO_ENTRADA included
            full_data = (entrance[0], codigo_entrada, entrance_date) + entrance[1:]
            data_to_insert.append(full_data)

        # Prepare the query
        insert_query = ''' 
        INSERT INTO ENTRADA_PLANTAS 
            (ESPECIE_LOTE_ID, CODIGO_ENTRADA, FECHA_ENTRADA, PROPAGACION,
            FECHA_SIEMBRA, CANTIDAD_SEMILLAS, FECHA_BROTACION,
            CANTIDAD_PLANTAS_EMBOLSADAS, FECHA_EMBOLSADO, FASE_LUNAR, NOTAS)
        VALUES (?,?,?,?,?,?,?,?,?,?,?)
        '''

        # Execute all inserts at once
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()
        
        print(f"✅ Successfully inserted {len(entrances)} plant entrances")
        return True
        
    except sqlite3.Error as e:
        print(f"❌ Database error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        if conn:
            conn.close()

# Example usage with corrected data format
# Format: (ESPECIE_LOTE_ID, PROPAGACION, FECHA_SIEMBRA, CANTIDAD_SEMILLAS, 
#          FECHA_BROTACION, CANTIDAD_PLANTAS_EMBOLSADAS, FECHA_EMBOLSADO, 
#          FASE_LUNAR, NOTAS)
new_entrances = [
    (5, 'Semilla-germinado en casa', '26-06-25', 30, '02-07-25', 13, '29-09-25', 'Cuarto creciente', 'Embolsado tardío con hojas caídas y color amarillo')
]

# Fixed function call - use consistent function name
log_entrance(new_entrances, '29-09-25')

✅ Successfully inserted 1 plant entrances


True

_____________
# Rain Tracking

As part of the irrigation processes, we consider and document rain as a collaborative force

In [3]:
import sqlite3 

# Rain data
rain_data =[
    ('2025-09-30 19:30', '2025-09-30 22:40', 'Llovizna Persistente', 'Llovió intensamente por un lapso de 40 minutos')
]

# Parametrized query 
insert_query = '''
    INSERT INTO REGISTRO_CLIMA 
        (FECHA_HORA_INICIO, FECHA_HORA_FIN, CATEGORIA_PRECIPITACION, NOTAS)
    VALUES (?,?,?,?)
'''

# Execute with verification
with sqlite3.connect('nursery.db') as conn:
    cursor = conn.cursor()
    cursor.executemany(insert_query, rain_data)

    # Basic verification
    rows_inserted = cursor.rowcount
    print(f"Rows inserted: {rows_inserted}")

    # Verify by counting final records
    cursor.execute("SELECT COUNT(*) FROM REGISTRO_CLIMA")
    total_records = cursor.fetchone()[0]
    print(f"Total records in table: {total_records}")

Rows inserted: 1
Total records in table: 12


In [8]:
import sqlite3

def delete_climate_record(date_time):
    try:
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()

        # Use parameterized query to avoid SQL injection and formatting issues
        cursor.execute(
            '''
            DELETE FROM REGISTRO_CLIMA
            WHERE FECHA_HORA_INICIO = ?
            ''',
            (date_time,)
        )
        
        rows_deleted = cursor.rowcount
        conn.commit()
        
        print(f"Rows deleted: {rows_deleted}")
        return rows_deleted
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
        return 0
    except Exception as e:
        print(f"General error: {e}")
        return 0
    finally:
        if conn:
            conn.close()

# Usage
delete_climate_record('2025-09-26 13:30')

Rows deleted: 0


0

______
# Pest Control Tracking